# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def func_query_janela(lista_janela, feat_num_lista, id, safra_ref, target, tb_feat, safra):
    query_janela = ""
    lista_vars = ""

    for n in lista_janela:
        query_variaveis_numericas=""
        for i in feat_num_lista:
            query_variaveis_numericas +=f"""
             -- Criação de variáveis numéricas a partir da coluna {i} para a janela {n}
            SUM(COALESCE({tb_feat}.{i},0)) AS {i}_SUM_{n}M,
            MIN(COALESCE({tb_feat}.{i},0)) AS {i}_MIN_{n}M,
            MAX(COALESCE({tb_feat}.{i},0)) AS {i}_MAX_{n}M,
            AVG(COALESCE({tb_feat}.{i},0)) AS {i}_AGV_{n}M,
            """
            lista_vars += f"""
            tb_join.{i}_SUM_{n}M,
            tb_join.{i}_MIN_{n}M,
            tb_join.{i}_MAX_{n}M,
            tb_join.{i}_AGV_{n}M,
            """
        query_variaveis_numericas = query_variaveis_numericas.rstrip(', \n')

        query_janela += f"""
        -- Criação de variáveis de janela de {n}M
        tb_janela_{n}M as(
            SELECT 
                tb_public.{id},
                tb_public.{safra_ref},
                tb_public.{target},
                {query_variaveis_numericas}

            FROM tb_public
            LEFT JOIN {tb_feat} 
            ON  tb_public.{id} = tb_feat.{id}
            AND (strftime('%Y-%m-%d', date({tb_feat}.{safra}, '+{n} months')) >= tb_public.{safra_ref})
                AND ({tb_feat}.{safra} < tb_public.{safra_ref})
            GROUP BY tb_public.{id}, tb_public.{safra_ref}
        ),
        """
    lista_vars = lista_vars.rstrip(', \n')
    return query_janela, lista_vars

In [ ]:
#| export
def func_query_join(lista_janela, id, safra_ref):
    query_join= ""
    for i in lista_janela:
        query_join += f"""
        LEFT JOIN tb_janela_{i}M
            ON tb_public.{id} = tb_janela_{i}M.{id}
            AND tb_public.{safra_ref} = tb_janela_{i}M.{safra_ref}
    """
    return query_join

In [ ]:
#| export
def func_query_final(tb_publico, lista_janela,feat_num_lista, id, safra_ref, target, tb_feat, safra):
    
    query_janela, lista_var = func_query_janela(lista_janela, feat_num_lista, id, safra_ref, target, tb_feat, safra)
    query_final = f"""
    with 
    tb_public as (
        SELECT 
            *
        FROM {tb_publico}
    ),
    {query_janela}

    tb_join as (
        SELECT 
            *
        FROM tb_public 
        {func_query_join(lista_janela, id, safra_ref)}
    )
        
    SELECT 
        tb_join.{id},
        tb_join.{safra_ref},
        tb_join.{target},
        {lista_var}
    FROM tb_join
    """
    return query_final

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()